Cell 1: Imports

In [1]:
import json
import torch
from torch.utils.data import Dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
import matplotlib.pyplot as plt
import pandas as pd

c:\Users\rejit\Development\ai\finacle-gpt2-finetune\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'matplotlib'

Cell 2: Dataset Class Definition

In [ ]:
class FinacleScriptDataset(Dataset):
    def __init__(self, file_path, tokenizer):
        self.tokenizer = tokenizer
        self.examples = []
        with open(file_path, "r") as f:
            data = json.load(f)
        for item in data:
            text = f"PROMPT: {item['prompt']}\nSCRIPT: {item['script']}{self.tokenizer.eos_token}"
            tokenized_text = self.tokenizer(text, truncation=True, max_length=128)
            self.examples.append(tokenized_text)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

Cell 3: Load Model and Tokenizer

In [ ]:
print("Loading model and tokenizer...")
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
print("Done.")

Cell 4: Prepare Dataset and Data Collator

In [ ]:
print("Preparing dataset...")
train_file_path = "training_data.json"
train_dataset = FinacleScriptDataset(file_path=train_file_path, tokenizer=tokenizer)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)
print("Done.")

Cell 5: Define Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./fine-tuned-model",
    num_train_epochs=30,
    per_device_train_batch_size=1,
    learning_rate=5e-6,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=15,
    dataloader_pin_memory=False,
)

Cell 6: Initialize and Run the Trainer

This is the cell that will perform the training. It will take a few moments to run.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=None,
    data_collator=data_collator,
)

print("Starting the fine-tuning process...")
trainer.train()
print("Training complete!")

Cell 7: Save the Final Model

In [ ]:
print("Saving the final model...")
trainer.save_model("./fine-tuned-model")
print("Model saved successfully.")

Cell 8: Analyze and Plot Training Loss 📈

This final cell will parse the training logs and generate a plot to visualize how the model's loss decreased over time.

In [ ]:
# The training history is stored in trainer.state.log_history
log_history = trainer.state.log_history

# Filter out the final summary metric to only get per-step logs
training_logs = [log for log in log_history if 'loss' in log]

# Create a pandas DataFrame for easier plotting
df = pd.DataFrame(training_logs)

# Create the plot
plt.style.use('seaborn-v0_8-whitegrid') # Use a nice style for the plot
plt.figure(figsize=(12, 7))
plt.plot(df['epoch'], df['loss'], marker='o', linestyle='-', color='b', label='Training Loss')

# Add titles and labels for clarity
plt.title('Training Loss Over Epochs', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.xticks(range(0, int(df['epoch'].max()) + 2)) # Ensure integer ticks for epochs
plt.legend()
plt.show()

Cell 9: Load the Fine-Tuned Model

This cell loads the model and tokenizer that you saved in the previous step. You only need to run this once after training.

In [ ]:
# --- Load the fine-tuned model and tokenizer from disk ---

print("Loading the fine-tuned model...")
model_path = './fine-tuned-model'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# If you are using a GPU, move the model to the GPU
if torch.cuda.is_available():
    model.to('cuda')

print("Model loaded successfully.")

Cell 10: Define a Reusable Generation Function
This cell defines a handy function to generate a script from any prompt. This makes it easy to test many different prompts without rewriting code.

In [ ]:
def generate_finacle_script(prompt_text):
    """
    This function takes a text prompt and generates a Finacle script
    using the loaded fine-tuned model.
    """
    # Format the prompt exactly as we did during training
    formatted_prompt = f"PROMPT: {prompt_text}\nSCRIPT:"

    # Tokenize the formatted prompt and move tensors to the correct device (GPU/CPU)
    inputs = tokenizer(formatted_prompt, return_tensors='pt')
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    
    print(f"\n---> Generating script for prompt: '{prompt_text}'")
    
    # Use the model's generate() method with sampling for more creative results
    # Note: We are not using beam search (num_beams) here to allow temperature to work.
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=150,
        do_sample=True,          # Activate sampling
        temperature=0.7,         # Make output less random
        top_k=50,                # Consider only the top 50 most likely tokens
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode the generated token IDs back into text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Clean up the output to only show the generated script
    try:
        script_part = generated_text.split("SCRIPT:")[1].strip()
    except IndexError:
        script_part = "Could not generate a valid script."
        
    # Print the result in a formatted way
    print("\n--- Generated Finacle Script ---")
    print(script_part)
    print("--------------------------------\n")

Cell 11: Test the Generation Function
Now you can use the function from the previous cell to test any prompt you like. Simply call the function with your desired prompt as the argument.

In [ ]:
# --- Example 1 ---
generate_finacle_script("Write a script that subtracts 5 from 20 and prints the result.")

# --- Example 2 ---
generate_finacle_script("Create a while loop that prints numbers from 1 to 3.")

# --- Example 3 (You can add your own) ---
generate_finacle_script("Convert the string 'FINACLE' to lowercase.")